In [1]:
import numpy as np
import random
import heapq
import time

In [2]:
class GridGame:
    def __init__(self, grid_size=5, initial_scores=10):
        self.grid_size = grid_size
        self.initial_scores = initial_scores
        self.grid = np.zeros((grid_size, grid_size))
        self.populate_initial_scores()

        self.players = {'A': (0, 0), 'B': (grid_size-1, grid_size-1)}
        self.scores = {'A': 0, 'B': 0}

    def populate_initial_scores(self):
        placed_scores = set()
        while len(placed_scores) < self.initial_scores:
            x, y = random.randint(0, self.grid_size-1), random.randint(0, self.grid_size-1)
            if (x, y) not in placed_scores:
                self.grid[x][y] = random.randint(1, 4)
                placed_scores.add((x, y))

    def add_random_score(self):
        x, y = random.randint(0, self.grid_size-1), random.randint(0, self.grid_size-1)
        while self.grid[x][y] != 0:
            x, y = random.randint(0, self.grid_size-1), random.randint(0, self.grid_size-1)
        self.grid[x][y] = random.randint(1, 4)

    def move_player(self, player, direction):
        x, y = self.players[player]
        if direction == 'up' and x > 0:
            x -= 1
        elif direction == 'down' and x < self.grid_size - 1:
            x += 1
        elif direction == 'left' and y > 0:
            y -= 1
        elif direction == 'right' and y < self.grid_size - 1:
            y += 1

        self.players[player] = (x, y)
        self.scores[player] += self.grid[x][y]
        self.grid[x][y] = 0

    def get_possible_moves(self, player):
        x, y = self.players[player]
        moves = []
        if x > 0:
            moves.append('up')
        if x < self.grid_size - 1:
            moves.append('down')
        if y > 0:
            moves.append('left')
        if y < self.grid_size - 1:
            moves.append('right')
        return moves

    def heuristic(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def a_star_move(self, player):
        start = self.players[player]
        goal = max(((i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.grid[i][j] > 0),
                   key=lambda x: self.grid[x[0]][x[1]], default=start)

        if start == goal:
            return

        frontier = []
        heapq.heappush(frontier, (0, start))
        came_from = {start: None}
        cost_so_far = {start: 0}

        while frontier:
            _, current = heapq.heappop(frontier)

            if current == goal:
                break

            for move in self.get_possible_moves(player):
                x, y = current
                if move == 'up':
                    next_pos = (x-1, y)
                elif move == 'down':
                    next_pos = (x+1, y)
                elif move == 'left':
                    next_pos = (x, y-1)
                elif move == 'right':
                    next_pos = (x, y+1)

                new_cost = cost_so_far[current] + 1
                if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                    cost_so_far[next_pos] = new_cost
                    priority = new_cost + self.heuristic(goal, next_pos)
                    heapq.heappush(frontier, (priority, next_pos))
                    came_from[next_pos] = current

        current = goal
        while came_from[current] != start:
            current = came_from[current]

        direction = None
        if current[0] < start[0]:
            direction = 'up'
        elif current[0] > start[0]:
            direction = 'down'
        elif current[1] < start[1]:
            direction = 'left'
        elif current[1] > start[1]:
            direction = 'right'

        if direction:
            self.move_player(player, direction)

    def dfs_move(self, player):
        start = self.players[player]
        stack = [start]
        visited = set()
        came_from = {start: None}

        while stack:
            current = stack.pop()

            if current in visited:
                continue

            visited.add(current)

            for move in self.get_possible_moves(player):
                x, y = current
                if move == 'up':
                    next_pos = (x-1, y)
                elif move == 'down':
                    next_pos = (x+1, y)
                elif move == 'left':
                    next_pos = (x, y-1)
                elif move == 'right':
                    next_pos = (x, y+1)

                if next_pos not in visited:
                    stack.append(next_pos)
                    came_from[next_pos] = current

        # 가장 높은 점수가 있는 곳으로 이동
        goal = max(visited, key=lambda pos: self.grid[pos[0]][pos[1]], default=start)

        current = goal
        while came_from[current] != start:
            current = came_from[current]

        direction = None
        if current[0] < start[0]:
            direction = 'up'
        elif current[0] > start[0]:
            direction = 'down'
        elif current[1] < start[1]:
            direction = 'left'
        elif current[1] > start[1]:
            direction = 'right'

        if direction:
            self.move_player(player, direction)

    def bfs_move(self, player):
        start = self.players[player]
        queue = [start]
        visited = set()
        came_from = {start: None}

        while queue:
            current = queue.pop(0)

            if current in visited:
                continue

            visited.add(current)

            for move in self.get_possible_moves(player):
                x, y = current
                if move == 'up':
                    next_pos = (x-1, y)
                elif move == 'down':
                    next_pos = (x+1, y)
                elif move == 'left':
                    next_pos = (x, y-1)
                elif move == 'right':
                    next_pos = (x, y+1)

                if next_pos not in visited:
                    queue.append(next_pos)
                    came_from[next_pos] = current

        # 가장 높은 점수가 있는 곳으로 이동
        goal = max(visited, key=lambda pos: self.grid[pos[0]][pos[1]], default=start)

        current = goal
        while came_from[current] != start:
            current = came_from[current]

        direction = None
        if current[0] < start[0]:
            direction = 'up'
        elif current[0] > start[0]:
            direction = 'down'
        elif current[1] < start[1]:
            direction = 'left'
        elif current[1] > start[1]:
            direction = 'right'

        if direction:
            self.move_player(player, direction)

    def dijkstra_move(self, player):
        start = self.players[player]
        goal = max(((i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.grid[i][j] > 0),
                   key=lambda x: self.grid[x[0]][x[1]], default=start)

        if start == goal:
            return

        frontier = []
        heapq.heappush(frontier, (0, start))
        came_from = {start: None}
        cost_so_far = {start: 0}

        while frontier:
            _, current = heapq.heappop(frontier)

            for move in self.get_possible_moves(player):
                x, y = current
                if move == 'up':
                    next_pos = (x-1, y)
                elif move == 'down':
                    next_pos = (x+1, y)
                elif move == 'left':
                    next_pos = (x, y-1)
                elif move == 'right':
                    next_pos = (x, y+1)

                new_cost = cost_so_far[current] + 1
                if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                    cost_so_far[next_pos] = new_cost
                    heapq.heappush(frontier, (new_cost, next_pos))
                    came_from[next_pos] = current

        current = goal
        while came_from[current] != start:
            current = came_from[current]

        direction = None
        if current[0] < start[0]:
            direction = 'up'
        elif current[0] > start[0]:
            direction = 'down'
        elif current[1] < start[1]:
            direction = 'left'
        elif current[1] > start[1]:
            direction = 'right'

        if direction:
            self.move_player(player, direction)

    def greedy_move(self, player):
        x, y = self.players[player]
        max_score = -1
        best_move = None
        for move in self.get_possible_moves(player):
            if move == 'up' and self.grid[x-1][y] > max_score:
                max_score = self.grid[x-1][y]
                best_move = move
            elif move == 'down' and self.grid[x+1][y] > max_score:
                max_score = self.grid[x+1][y]
                best_move = move
            elif move == 'left' and self.grid[x][y-1] > max_score:
                max_score = self.grid[x][y-1]
                best_move = move
            elif move == 'right' and self.grid[x][y+1] > max_score:
                max_score = self.grid[x][y+1]
                best_move = move

        if best_move:
            self.move_player(player, best_move)

    def random_move(self, player):
        move = random.choice(self.get_possible_moves(player))
        self.move_player(player, move)

    def print_game_state(self, turn):
        print(f"Turn {turn}")
        print("Grid state:")
        for row in self.grid:
            print(" ".join(f"{int(val):2d}" for val in row))
        print(f"Player A position: {self.players['A']}, Score: {self.scores['A']}")
        print(f"Player B position: {self.players['B']}, Score: {self.scores['B']}")
        print("\n")

    def simulate_game(self, turns=100, print_grid=True, algo_A='greedy', algo_B='random'):
        # 초기 점수 설정을 시뮬레이션 시작 시 다시 초기화
        self.grid = np.zeros((self.grid_size, self.grid_size))
        self.populate_initial_scores()
        self.players = {'A': (0, 0), 'B': (self.grid_size-1, self.grid_size-1)}
        self.scores = {'A': 0, 'B': 0}

        # Turn 0 상태 출력
        if print_grid:
            self.print_game_state(0)

        algo_map = {
            'greedy': self.greedy_move,
            'random': self.random_move,
            'a_star': self.a_star_move,
            'dfs': self.dfs_move,
            'bfs': self.bfs_move,
            'dijkstra': self.dijkstra_move
        }

        for turn in range(turns):
            self.add_random_score()
            algo_map[algo_A]('A')
            algo_map[algo_B]('B')
            if print_grid:
                self.print_game_state(turn + 1)

        return self.scores

In [3]:
def run_simulation(turns, print_grid, algo_A, algo_B):
    game = GridGame()
    result = game.simulate_game(turns=turns, print_grid=print_grid, algo_A=algo_A, algo_B=algo_B)
    return result

def multiple_simulations(runs, turns, print_grid, algo_A, algo_B):
    a_wins = 0
    b_wins = 0
    total_time = 0

    for _ in range(runs):
        start_time = time.time()
        result = run_simulation(turns, print_grid, algo_A, algo_B)
        end_time = time.time()

        total_time += (end_time - start_time)

        if result['A'] > result['B']:
            a_wins += 1
        elif result['B'] > result['A']:
            b_wins += 1

    total_games = a_wins + b_wins
    if total_games > 0:
        a_win_rate = a_wins / total_games * 100
        b_win_rate = b_wins / total_games * 100
    else:
        a_win_rate = b_win_rate = 0

    avg_time_per_simulation = total_time / runs

    print(f"Player A ({algo_A}) win rate: {a_win_rate:.2f}%")
    print(f"Player B ({algo_B}) win rate: {b_win_rate:.2f}%")
    print(f"Draw rate: {100 - a_win_rate - b_win_rate:.2f}%")
    print(f"Average time per simulation: {avg_time_per_simulation:.4f} seconds")

# 모든 알고리즘 조합을 테스트하는 함수
def test_all_algorithms(runs, turns, print_grid):
    algorithms = ['greedy', 'random', 'a_star']

    for algo_A in algorithms:
        for algo_B in algorithms:
            print(f"Testing: Player A ({algo_A}) vs Player B ({algo_B})")
            multiple_simulations(runs=runs, turns=turns, print_grid=print_grid, algo_A=algo_A, algo_B=algo_B)
            print("\n")


In [ ]:
# algo: {'greedy', 'random', 'a_star', 'dfs', 'bfs', 'dijkstra'}
run_simulation(turns=10, print_grid=False, algo_A='a_star', algo_B='dfs')

In [5]:
multiple_simulations(runs=100, turns=20, print_grid=False, algo_A='a_star', algo_B='greedy')

Player A (a_star) win rate: 38.14%
Player B (greedy) win rate: 61.86%
Draw rate: 0.00%
Average time per simulation: 0.0010 seconds


In [13]:
test_all_algorithms(runs=100, turns=20, print_grid=False)

Testing: Player A (greedy) vs Player B (greedy)
Player A (greedy) win rate: 57.58%
Player B (greedy) win rate: 42.42%
Draw rate: 0.00%
Average time per simulation: 0.0004 seconds


Testing: Player A (greedy) vs Player B (random)
Player A (greedy) win rate: 96.97%
Player B (random) win rate: 3.03%
Draw rate: 0.00%
Average time per simulation: 0.0003 seconds


Testing: Player A (greedy) vs Player B (a_star)
Player A (greedy) win rate: 67.68%
Player B (a_star) win rate: 32.32%
Draw rate: -0.00%
Average time per simulation: 0.0009 seconds


Testing: Player A (random) vs Player B (greedy)
Player A (random) win rate: 3.03%
Player B (greedy) win rate: 96.97%
Draw rate: 0.00%
Average time per simulation: 0.0003 seconds


Testing: Player A (random) vs Player B (random)
Player A (random) win rate: 52.13%
Player B (random) win rate: 47.87%
Draw rate: -0.00%
Average time per simulation: 0.0003 seconds


Testing: Player A (random) vs Player B (a_star)
Player A (random) win rate: 10.42%
Player B (a_